In [1]:
import torch
from transformers import pipeline

In [2]:
test="""
La biodiversité est un élément vital de notre planète. Elle englobe la variété de la vie sur Terre, des micro-organismes aux plantes, aux animaux et aux écosystèmes complexes. La biodiversité joue un rôle essentiel dans la stabilité et la résilience de nos écosystèmes, ainsi que dans notre propre survie en fournissant des services écosystémiques tels que la pollinisation des cultures, la purification de l'eau et la régulation du climat.

Cependant, la biodiversité est actuellement confrontée à de graves menaces. La perte d'habitats due à la déforestation, à l'urbanisation et à l'agriculture intensive réduit les zones où les espèces peuvent survivre. La surpêche et la pollution des océans mettent en danger les écosystèmes marins. Le changement climatique perturbe les habitats naturels et pousse de nombreuses espèces vers l'extinction.

La conservation de la biodiversité est donc une préoccupation mondiale importante. Il est essentiel de protéger les espèces en danger, de restaurer les écosystèmes endommagés et de promouvoir une gestion durable des ressources naturelles. Les efforts de conservation peuvent contribuer à maintenir la diversité biologique de notre planète pour les générations futures et à préserver l'équilibre délicat de la vie sur Terre.
"""

In [3]:
assistant_message = """
Tu est un expert en NLP, tu as pour tâche d'extraire les triples (sujet, prédicat, objet) présenté dans des phrases données. 
Voici un exemple :
Phrase : Le chat noir dort paisiblement sur le canapé.
Subjet: Le chat noir
Prédicat: dort
Objet: sur le canapé
Voici un autre exemple :
Phrase :  Les chercheurs ont découvert une nouvelle espèce de papillon, qui vit exclusivement dans les forêts tropicales humides.
Subjet: Les chercheurs
Prédicat: ont découvert
Objet: une nouvelle espèce de papillon
"""

# Zephr

In [1]:
import torch
from transformers import pipeline, BitsAndBytesConfig,AutoModelForCausalLM, AutoTokenizer

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

model_name = "HuggingFaceH4/zephyr-7b-alpha"
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

generation_config = model.generation_config
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = True
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


extraction triplets

In [2]:
import torch
from transformers import pipeline, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", quantization_config=quantization_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": assistant_message,
    },
    {"role": "user", "content": f"Donne moi tous les triples (sujet, prédicat, objet) présenté dans ce paragraphe : {test} ?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/home/ubuntu/miniconda3/envs/finetune_venv/lib/python3.11/site-packages/transformers/generation/utils.py:1462: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>

Tu est un expert en NLP, tu as pour tâche d'extraire les triples (sujet, prédicat, objet) présenté dans des phrases données. 
Voici un exemple :
Phrase : Le chat noir dort paisiblement sur le canapé.
Subjet: Le chat noir
Prédicat: dort
Objet: sur le canapé
Voici un autre exemple :
Phrase :  Les chercheurs ont découvert une nouvelle espèce de papillon, qui vit exclusivement dans les forêts tropicales humides.
Subjet: Les chercheurs
Prédicat: ont découvert
Objet: une nouvelle espèce de papillon
</s>
<|user|>
Donne moi tous les triples (sujet, prédicat, objet) présenté dans ce paragraphe : 
La biodiversité est un élément vital de notre planète. Elle englobe la variété de la vie sur Terre, des micro-organismes aux plantes, aux animaux et aux écosystèmes complexes. La biodiversité joue un rôle essentiel dans la stabilité et la résilience de nos écosystèmes, ainsi que dans notre propre survie en fournissant des services écosystémiques tels que la pollinisation des cultures, la pu

Simple question about fine tuning

In [4]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {"role": "user", "content": "Dis moi tout ce que tu sais sur tremblement de terre."},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


<|user|>
Dis moi tout ce que tu sais sur tremblement de terre.</s>
<|assistant|>
Je suis désolé, mais je ne suis pas capable de vous dire ce que je sais sur le tremblement de terre en français, car je suis un robot de langage. Cependant, j'ai des informations et des connaissances sur les tremblements de terre que je pourrais vous partager.

Un tremblement de terre est un événement naturel causé par la libération soudaine d'énergie dans la croûte terrestre. Les tremblements de terre peuvent être dangereux et causent souvent de lourds dégâts matériels et humains. Les tremblements de terre sont classés en fonction de leur magnitude, qui mesure leur intensité. La magnitude la plus courante est la magnitude de moment, qui est mesurée sur l'échelle de Richter.

Les tremblements de terre sont causés par des mouvements de plaques tectoniques, qui sont des blocs de roche qui composent la croûte terrestre. Les plaques tectoniques se déplacent lent


# Mistral 7b instruct

In [4]:
import torch
from transformers import pipeline, BitsAndBytesConfig,AutoModelForCausalLM, AutoTokenizer

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

generation_config = model.generation_config
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = True
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def chat_completion(messages):
    prompt = ""
    for m in messages:
        if m["role"]=="user":
            prompt += "[INST]" +  m["content"] + "[/INST]"
        elif m["role"]=="assistant":
            prompt += "<s>" + m["content"] + "</s>"
    
    return prompt.strip()

In [6]:
messages = [
    {"role": "assistant", "content": assistant_message},
    {"role": "user", "content": f"Donne moi tous les triples (sujet, prédicat, objet) présenté dans ce paragraphe : {test} ?"}
]
completed_prompt = chat_completion(messages)
completed_prompt

"<s>\nTu est un expert en NLP, tu as pour tâche d'extraire les triples (sujet, prédicat, objet) présenté dans des phrases données. \nVoici un exemple :\nPhrase : Le chat noir dort paisiblement sur le canapé.\nSubjet: Le chat noir\nPrédicat: dort\nObjet: sur le canapé\nVoici un autre exemple :\nPhrase :  Les chercheurs ont découvert une nouvelle espèce de papillon, qui vit exclusivement dans les forêts tropicales humides.\nSubjet: Les chercheurs\nPrédicat: ont découvert\nObjet: une nouvelle espèce de papillon\n</s>[INST]Donne moi tous les triples (sujet, prédicat, objet) présenté dans ce paragraphe : \nLa biodiversité est un élément vital de notre planète. Elle englobe la variété de la vie sur Terre, des micro-organismes aux plantes, aux animaux et aux écosystèmes complexes. La biodiversité joue un rôle essentiel dans la stabilité et la résilience de nos écosystèmes, ainsi que dans notre propre survie en fournissant des services écosystémiques tels que la pollinisation des cultures, la 

In [9]:
text = pipe(
    completed_prompt,
    temperature=0.1
    )[0]
print(text)

{'generated_text': ' Voila toutes les tripletes extraits du texte ci dessus avec leur sujets, predicateurs et objets respectifs :\n1. Suject - Biodiversity ; Précipitateur – is an element; Objet – of our planet2. SubJECTS - Micro organisms, plants and animals3. SUBJEKTS - Ecosystems4. OBJECTIVE - plays a role essential5. SUBUGETTEES - in the stability6. SUPUBETEESSSES - survival7. SERVICES - ecoystematic8. THREATENINGS - loss9. OF HABITAITS - through deforestaions0. DEFORESTATION AND URBANIZATION IN AGRICULTURE INTENSIFICATION REDUCE THE ZONES WHERE ESPERENCHE CAN LIVING. OTHERS ARE PUT AT RISKS BY FISHHUNGHT AN DPOLLUITIONOFT HE WATEFURTHERMOAR THOUGHS CLIMACT CHANGED NATRUAL Habitatss THAT FORCISED MANY SPEECLES TO EXTINCTIONSUCH AS WHALE SHARK'}


In [10]:
text["generated_text"]

' Voila toutes les tripletes extraits du texte ci dessus avec leur sujets, predicateurs et objets respectifs :\n1. Suject - Biodiversity ; Précipitateur – is an element; Objet – of our planet2. SubJECTS - Micro organisms, plants and animals3. SUBJEKTS - Ecosystems4. OBJECTIVE - plays a role essential5. SUBUGETTEES - in the stability6. SUPUBETEESSSES - survival7. SERVICES - ecoystematic8. THREATENINGS - loss9. OF HABITAITS - through deforestaions0. DEFORESTATION AND URBANIZATION IN AGRICULTURE INTENSIFICATION REDUCE THE ZONES WHERE ESPERENCHE CAN LIVING. OTHERS ARE PUT AT RISKS BY FISHHUNGHT AN DPOLLUITIONOFT HE WATEFURTHERMOAR THOUGHS CLIMACT CHANGED NATRUAL Habitatss THAT FORCISED MANY SPEECLES TO EXTINCTIONSUCH AS WHALE SHARK'